### Análise dos Covenants Contábeis divulgados pelas empresas listadas na B# (2010 a 2022)

In [29]:
## bibliotecas usadas
import pandas as pd
import numpy as np
import unicodedata
import os
import re
from fuzzywuzzy import fuzz, process


m:\apps no sd\Python\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### A análise começou com a coleta manual das notas explicativas publicadas pelas empresas no site da B3 Investidor, como também da Comissão de Valores Mobiliários (CVM).
Após coleta fiz um script para ler todos os pdfs (disponível no github: `link`) e separar em uma pasta todos que citaram covenants ou cláusulas restritivas.

In [20]:
empresas_df = pd.read_excel("M:\{estudos}\Python\Covenants-Contabeis\Empresas_com_covenants.xlsx")
empresas_df.head()

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_10064\62090819.py:1: SyntaxWarning: invalid escape sequence '\{'
  empresas_df = pd.read_excel("M:\{estudos}\Python\Covenants-Contabeis\Empresas_com_covenants.xlsx")


,Empresa,Quantidade de Anos
0,Hidrovias do Brasil,11
1,Ambipar Participacoes e Empreendimentos,4
2,Plano & Plano Desenvolvimento Imobiliario SA,4
3,Renner,13
4,B3 SA Brasil Bolsa Balcao,3


In [21]:
empresas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Empresa             282 non-null    object
 1   Quantidade de Anos  282 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 4.5+ KB


## Dividindo agora a base de dados da análise manual de todas as notas explicativas que citaram "covenants"

In [22]:
covenants = pd.read_excel("M:\{estudos}\Python\Covenants-Contabeis\Base Covenants Contabeis.xlsx")
covenants.tail()

<>:1: SyntaxWarning: invalid escape sequence '\{'
<>:1: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Maria Eduarda\AppData\Local\Temp\ipykernel_10064\3345557802.py:1: SyntaxWarning: invalid escape sequence '\{'
  covenants = pd.read_excel("M:\{estudos}\Python\Covenants-Contabeis\Base Covenants Contabeis.xlsx")


,EMPRESA,ANO,POSSUI COVENANT,DIVULGOU,Debenture ou Empréstimos e financiamento,Índice Utilizado,Limite,Violou?
6266,Zamp,2021.0,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,SIM
6267,Zamp,2022.0,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO
6268,Zamp,2022.0,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO
6269,Zamp,2023.0,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO
6270,Zamp,2023.0,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO


In [23]:
covenants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6271 entries, 0 to 6270
Data columns (total 8 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   EMPRESA                                   6264 non-null   object 
 1   ANO                                       6254 non-null   float64
 2   POSSUI COVENANT                           6254 non-null   object 
 3   DIVULGOU                                  5979 non-null   object 
 4   Debenture ou Empréstimos e financiamento  5238 non-null   object 
 5   Índice Utilizado                          5197 non-null   object 
 6   Limite                                    4822 non-null   object 
 7   Violou?                                   4620 non-null   object 
dtypes: float64(1), object(7)
memory usage: 392.1+ KB


In [24]:
covenants.dropna(subset=['ANO'], inplace=True)
covenants['ANO'] = covenants['ANO'].astype(int)

# limpeza coluna 'Limite'
valores_unicos = covenants['Limite'].dropna().unique()
valores_unicos

array(['menor ou igual a 3,5', 'menor ou igual a 4,5',
       'maior ou igual a 1,25', 'maior ou igual a 1,2',
       'maior ou igual a 1,3', 'maior ou igual a 1', 'menor ou igual a 3',
       'maior ou igual a 1,1', 'menor ou igual a 2,5',
       'maior ou igual a 1,75', 'menor ou igual a 3,6',
       'maior ou igual a 1,76', 'maior ou igual a 1,5',
       'menor ou igual a 3,75', 'maior ou igual a 1,10',
       'menor ou igual a 5,6', 'maior ou igual a 1,15',
       'menor ou igual a 1,9 bi', 'menor ou igual a 338 mi',
       'menor ou igual a 840 mi', 'menor ou igual a 285 mi',
       'maior ou igual a 20%', 'menor ou igual a 50 mi',
       'maior ou igual a 41 mi', 'menor ou igual a 164 mi',
       'menor ou igual a 225 mi', 'maior ou igual a 1,20',
       'maior ou igual a 3,75', 'menor ou igual a 1,75',
       'menor ou igual a 0,6', 'maior ou igual a 3,50',
       'maior ou igual a 0,6', 'menor ou igual a 2,75',
       'menor ou igual a 2,50', 'menor ou igual a 3,0',
       'men

In [26]:
# Função para normalizar o texto (remove acentos e converte para minúsculo)
def normalizar_texto(texto):
    texto = str(texto).lower().strip()
    texto = texto.replace(',', '.')
    texto = re.sub(r'\s+', ' ', texto)  # remove espaços duplos
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    return texto

# Lista de operadores em regex
operadores_regex = (
    r'(maior ou igual a|maior ou igual|menor ou igual a|menor ou igual|'
    r'maior que|menor que|igual a)'
)

# Função principal
def extrair_limite(texto):
    if pd.isnull(texto):
        return pd.Series([None, None])

    texto_original = texto  # para verificar se tem %
    texto = normalizar_texto(texto)

    # Ignorar se houver unidades monetárias ou palavras irrelevantes
    if any(unidade in texto for unidade in ['r$', 'milhao', 'milhoes', 'mi', 'pl', 'depreciacao', 'amortizacao']):
        return pd.Series([None, None])

    # Detectar porcentagem
    tem_porcentagem = '%' in texto_original

    # Regex para operador e valor
    padrao = fr'{operadores_regex}\s*([0-9.]+)'
    match = re.search(padrao, texto)

    if match:
        operador = match.group(1).strip()
        try:
            valor = float(match.group(2))
            if tem_porcentagem:
                valor = valor / 100
            return pd.Series([operador, valor])
        except:
            return pd.Series([operador, None])

    return pd.Series([None, None])

df[['operador_limite', 'valor_limite']] = df['Limite'].apply(extrair_limite)
df.tail()


,EMPRESA,ANO,POSSUI COVENANT,DIVULGOU,Debenture ou Empréstimos e financiamento,Índice Utilizado,Limite,Violou?,operador_limite,valor_limite
6266,Zamp,2021,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,SIM,menor ou igual a,3.0
6267,Zamp,2022,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO,menor ou igual a,3.0
6268,Zamp,2022,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO,menor ou igual a,3.0
6269,Zamp,2023,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO,menor ou igual a,3.0
6270,Zamp,2023,SIM,SIM,Debêntures,Dívida Líquida / EBITDA,menor ou igual a 3,NÃO,menor ou igual a,3.0


In [ ]:
df_empresas_setores = pd.read_excel("Setores das empresas B3.xlsx")
df_empresas_setores.columns = df_empresas_setores.columns.str.strip()
lista_nomes = df_empresas_setores["Nome da empresa no pregão"].dropna().tolist()
df_empresas_setores

,Nome da empresa no pregão,Ticker na bolsa,Setor
0,AERIS,AERI3,Bens Industriais
1,ARMAC,ARML3,Bens Industriais
2,ATMASA,ATMP3,Bens Industriais
3,AZEVEDO,AZEV4,Bens Industriais
4,AZUL,AZUL4,Bens Industriais
...,...,...,...
370,POLPAR,PPAR3,Outros
371,PROMPT PART,PRPT3,Outros
372,SUDESTE S/A,OPSE3,Outros
373,SUL 116 PART,OPTS3,Outros


In [32]:
df["EMPRESA"] = df["EMPRESA"].str.strip().str.upper()
df_empresas_setores["Nome da empresa no pregão"] = df_empresas_setores["Nome da empresa no pregão"].str.strip().str.upper()

def match_nome(nome, lista_nomes, threshold=60):
    melhor_match = process.extractOne(nome, lista_nomes, scorer=fuzz.partial_ratio)
    if melhor_match and melhor_match[1] >= threshold:
        return melhor_match[0]
    return None

# Aplica o fuzzy match e salva em nova coluna
df["Nome ajustado"] = df["EMPRESA"].apply(lambda x: match_nome(x, lista_nomes))

# Faz merge usando a coluna ajustada
df_merged = pd.merge(
    df,
    df_empresas_setores,
    left_on="Nome ajustado",
    right_on="Nome da empresa no pregão",
    how="left"
)

# Exporta e mostra os dados

df_merged.to_excel("empresas_merge_final.xlsx", index=False)
display(df_merged[["EMPRESA", "Nome ajustado", "Setor"]].head(10))

,EMPRESA,Nome ajustado,Setor
0,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS,NaN
1,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS,NaN
2,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS,NaN
3,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS,NaN
4,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS,NaN
5,AES BRASIL ENERGIA,AES BRASIL,NaN
6,AES BRASIL ENERGIA,AES BRASIL,NaN
7,AES BRASIL ENERGIA,AES BRASIL,NaN
8,AES BRASIL ENERGIA,AES BRASIL,NaN
9,AES BRASIL ENERGIA,AES BRASIL,NaN


In [33]:
nao_encontradas = df_merged[df_merged["Setor"].isna()][["EMPRESA", "Nome ajustado"]].drop_duplicates()
print("Empresas não encontradas:", nao_encontradas.shape[0])
display(nao_encontradas.head(10))


Empresas não encontradas: 259


,EMPRESA,Nome ajustado
0,AERIS INDUSTRIA E COMERCIO DE EQUIPAMENTOS PAR...,AERIS
5,AES BRASIL ENERGIA,AES BRASIL
37,AGRO TRENDS PARTICIPAÇÕES S.A.,TENDA
43,ALLIANCA SAUDE E PARTICIPACOES SA,INC SA
54,ALLIED TECNOLOGIA SA,ALLIED
59,ALLOS SA,KALLAS
84,ALLPARK EMPREENDIMENTOS PARTICIPACOES E SERVICOS,ALLIAR
97,ALPARGATAS,ALPARGATAS
111,ALPER CONSULTORIA E CORRETORA DE SEGUROS,VALE
112,ALPHAVILLE,ALPHAVILLE


## 🧠 1. **Descrição geral da base de dados**

**Objetivo**: contextualizar o leitor sobre sua amostra.

Você pode incluir:

* Número total de empresas e anos analisados.
* Distribuição temporal da presença de covenants (`"POSSUI COVENANT"`).
* Número e proporção de empresas que divulgaram (`"DIVULGOU"`).
* Frequência dos tipos de instrumentos (debêntures vs. empréstimos).
* Frequência de presença de cláusulas com limites e índices.

📌 *Gráficos sugeridos*: histogramas, barras, timeline com linhas empilhadas.

---

## 🏗️ 2. **Análise dos tipos de índices utilizados**

**Objetivo**: entender os indicadores financeiros mais usados como base para os covenants.

Analise:

* Frequência de cada valor em `"Índice Utilizado"` (ex: dívida/EBITDA, cobertura de juros, etc).
* Agrupamento por tipo de índice (liquidez, rentabilidade, endividamento, etc).
* Como esses índices variam por setor.

📌 *Gráficos*: gráfico de barras horizontais com os índices mais comuns.

---

## 📊 3. **Distribuição dos limites numéricos**

**Objetivo**: avaliar os **valores definidos nos contratos** como critérios para quebra de covenant.

* Distribuição dos valores numéricos extraídos (`valor_limite`) por índice.
* Diferença nos limites definidos por setor.
* Comparar limites para empresas que **violaram** e **não violaram**.

📌 *Gráficos*: boxplots por índice, por setor, por status de violação.

---

## 🚨 4. **Análise de violação**

**Objetivo**: identificar padrão de violação contratual.

* Percentual de empresas que violaram covenants (`"Violou?" == 'sim'`).
* Comparar com a média do índice (se tiver os valores reais dos indicadores).
* Quais índices têm maior taxa de violação?
* Existe associação entre tipo de índice e violação?

📌 *Gráficos*: barras empilhadas, heatmaps.

---

## 🏢 5. **Setores mais expostos a covenants**

**Com a variável `Setor` adicionada**, você pode:

* Ver a proporção de empresas por setor que têm cláusulas de covenant.
* Quais setores mais divulgam os termos?
* Quais setores têm limites mais rígidos?
* Quais setores mais violam cláusulas?

📌 *Gráficos*: barras, heatmaps, scatterplots de valor do limite × setor.

---

## 📅 6. **Evolução temporal**

**Se tiver a variável `ANO` bem preenchida**:

* Evolução do uso de covenants ao longo do tempo.
* Mudança nos limites definidos ano a ano.
* Tendência de divulgação e violação.

📌 *Gráficos*: linhas por ano, área acumulada, barras temporais.

---

## 🔍 7. **Casos qualitativos especiais**

Você pode mencionar ou classificar manualmente:

* Limites com condições não numéricas (ex: `"menor ou igual a 75% da depreciação"`).
* Cláusulas que envolvem múltiplos critérios (`"maior ou igual a 1,5 ou menor que 0"`).
* Casos com valor em R\$, milhões, ou pagamentos parcelados.

📌 *Sugestão*: construir uma tabela de exemplos e agrupar por "não padronizável", "qualitativo", "financeiro monetário", etc.

---

## 📚 8. **Discussão econométrica (opcional)**

Se tiver tempo e dados complementares:

* Você pode rodar uma regressão para **estimar a probabilidade de violação**, com variáveis como:

  * tipo de índice
  * setor
  * ano
  * valor do limite
  * tipo de instrumento (debênture vs empréstimo)

📌 *Modelo sugerido*: regressão logística.

---

## ✨ Exemplos de perguntas para seu artigo

* "Quais índices são mais usados como covenants nas empresas listadas da B3?"
* "Há setores que enfrentam cláusulas mais restritivas?"
* "Os limites definidos nos contratos mudaram com o tempo?"
* "As cláusulas são efetivas? Há alto índice de violação?"
* "Existe alguma evidência de que empresas com 
